In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [103]:
import pandas as pd
import numpy as np
import calendar
import datetime
import re
import os

def filter_data(input_file_path,keyword):
    file = os.listdir(input_file_path)[0]
    dataFrame= pd.read_csv(os.path.join(input_file_path, file))
    # select rows containing text "Affirm"
    data_filtered = dataFrame.loc[dataFrame['DESCRIPTION'].str.contains(keyword),['DATE', 'DESCRIPTION', 'RECEIVED']]
    
    #split the decription
    new =data_filtered.DESCRIPTION.str.split(expand=True)
    data_filtered['DEPOSIT']=new[15]
    data_filtered.drop(columns =["DESCRIPTION"], inplace = True)
    data_filtered['DESCRIPTION']=new[3]+' '+new[4]
   
    # split the deposit id
    data_filtered[['IID','DEPOSIT_ID']]=data_filtered.DEPOSIT.str.split(":",expand=True)
    data_filtered.drop(columns =["DEPOSIT"], inplace = True)
    data_filtered.drop(columns =["IID"], inplace = True)

    # print(data_filtered.to_string())
    return data_filtered

In [141]:
def merge_filtered(res_data,affirm_report):
   #read the affirm report file
   data_report=pd.read_csv(affirm_report)
   data_report.rename(columns = {'deposit_id':'DEPOSIT_ID'}, inplace = True)
   #drop duplicate based on order id
   new_dt=data_report.drop_duplicates(subset='order_id')
   #print(new_dt)
   #combine two csv files based on DEPOSIT ID
   df = pd.merge(res_data,new_dt,on="DEPOSIT_ID")
  
   df.rename(columns = {'RECEIVED':'BANK AMOUNT'}, inplace = True)
   df.rename(columns = {'order_id':'ORDER'}, inplace = True)
   df['FEE']=df['fees']+df['txn_fees']
   df['ORDER']=df['ORDER'].astype(str)
  # print(df.to_string())
   return df


In [144]:
def merge_order(data_report,order_report_table):
    report_table=pd.read_csv(order_report_table)   #read the order report table
    report_table.rename(columns = {'Order_Number':'ORDER'}, inplace = True)
    #convert the object type to float
    report_table['Item_Price']=report_table['Item_Price'].apply(lambda x: float(x.split()[0].replace(',', '')))
    report_table['Item_Discount']=report_table['Item_Discount'].apply(lambda x: float(x.split()[0].replace(',', '')))
    # print(report_table)
    #combine report table to df
    data_table=pd.merge(data_report,report_table,on='ORDER')
    data_table.rename(columns = {'Item_Price':'PRICE'}, inplace = True) # rename columns
    data_table.rename(columns = {'Item_Discount':'DISCOUNT'}, inplace = True) # rename columns
    data_table.rename(columns = {'Order_Date':'QBO DATE'}, inplace = True) # rename columns
    data_table['QBO Description']=data_table['ORDER']
    data_table['INCOME']=data_table['PRICE']

    #add new columns
    data_table['PRICE']=data_table['PRICE'].astype(float)
    data_table['DISCOUNT']=data_table['DISCOUNT'].astype(float)
    data_table['NET AMOUNT']=data_table['PRICE']+data_table['FEE']+data_table['DISCOUNT']
    
    #add a new columns check by comparing two columns
    data_table['CHECK']=np.where((data_table['NET AMOUNT'] == data_table['total_settled']), 'Correct', 'Incorrect')
    # print(data_table.to_string())
    
    #change the datatype to int
    data_table['ORDER']=data_table['ORDER'].astype(int)

    return data_table

In [260]:
def check_register(order_data,reg_report,output_file_path):
    current_date=datetime.datetime.now()
    registr_report=pd.read_csv(reg_report)
    order_data['Success'] = order_data['ORDER'].isin(registr_report['Order id']).astype(int)
    registr_report.rename(columns = {'Order id':'ORDER'}, inplace = True) # rename columns
    registr_report.rename(columns = {'Session Name':'SESSION'}, inplace = True) # rename columns

    #split the session Name to get the course month
    registr_report[['cl1','cl2','Month']]=registr_report.SESSION.str.split("|",expand=True)
    registr_report[['colmn0','colmn1','colmn2','Course Month','clmn4']]=registr_report.Month.str.split(expand=True)
    
    registr_report['Course Month']= pd.to_datetime(registr_report['Course Month'])
    registr_report['Account'] = 'N/A'
    registr_report.loc[registr_report['Course Month'] < current_date, 'Account'] = '4000 Tution Income'
    registr_report.loc[registr_report['Course Month'] > current_date, 'Account'] = '2200 Unearned Revenue'

    registr_report['Course Month']=registr_report['Course Month'].dt.strftime('%b %Y') 
    
   
    #trying to merge the files
    new_df=pd.merge(order_data,registr_report,on='ORDER',how='left')
    new_df['Course Month']=new_df['Course Month'].astype(object)
    new_df['Course Month']=np.where((new_df['Success'] == 1), new_df['Course Month'], 'N/A')
    new_df.loc[new_df['Course Month'] == 'N/A', 'Account'] = '4000 Tution Income'
   
    #create ne df from new_df with required colmn
    output_data=new_df[['DATE', 'DESCRIPTION','BANK AMOUNT','PRICE','FEE','DISCOUNT','NET AMOUNT','CHECK','Course Month','ORDER','QBO DATE','QBO Description','INCOME','Account']].copy()
    # print(output_data.dtypes)
    # print(output_data.to_string())
    #convert the output to csv file
    output_data.to_csv('affirm_report_summary2.csv', encoding = 'utf-8-sig') 
    report='affirm_report_summary2.csv'
    output_data.to_csv(os.path.join(output_file_path,report), index = False) 

In [261]:
data=filter_data('/content/drive/MyDrive/Colab Notebooks/input_file_path','AFFIRM')
res_data=merge_filtered(data,'/content/drive/MyDrive/Affirm Testing/Affirm_Report_0123.csv')
merge_data=merge_order(res_data,'/content/drive/MyDrive/Affirm Testing/order_report_Jan22_Jan23.csv')
output=check_register(merge_data,'/content/drive/MyDrive/Colab Notebooks/registration_report_Dec22 _Jan23  - Sheet1.csv','/content/drive/MyDrive/Colab Notebooks/output_file_path')
# print(output)